In [110]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import ast
import json
from os import listdir
from os.path import isfile, join

In [111]:
''' EXTRACT TIME RANGE '''
mypath = 'cropped_data_labeled/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
time_range = np.array(sorted([float(x[19:-5]) for x in list(filter(lambda s: 'json' in s, onlyfiles))]))
time_range = time_range[time_range >= 30]
time_range

array([ 30. ,  33. ,  36. ,  39. ,  42.1,  45. ,  48. ,  51. ,  54. ,
        57. ,  60. ,  63. ,  66. ,  69. ,  72. ,  75. ,  78. ,  81. ,
        84. ,  87. ,  90. ,  93. ,  96. ,  99. , 102. , 105. , 108. ,
       111. , 114. , 117. , 120. , 123. , 126. , 129. , 132. , 135. ,
       138. , 141. , 144. , 147. , 150. , 153. , 156. , 159. , 162. ,
       165. , 168. , 171. , 174. , 177. , 180. , 183. , 186. , 189. ,
       192. , 195. , 198. , 201. , 204. , 207. , 210.1, 214. , 217. ,
       220. , 223. , 226. , 229. , 232. , 235. , 238. , 241. , 244. ,
       247. , 250. , 253. , 256. , 259. , 262. , 265. , 268. , 271. ,
       274. , 277. , 280. , 283. , 286. , 289. , 292. , 295. , 298. ,
       301. , 304. , 307. , 310. , 313. , 316. , 319. , 322. , 325. ,
       328. ])

In [85]:
detected_vortices = [(230,510), (100,200), (1000,1050), (540,1050), (549,740)]

In [86]:
frame = cv2.imread(f'cropped_data_labeled/x_vel_cropped_time={np.round(t,1)}0.png')
for x,y in detected_vortices:
    cv2.rectangle(frame, (x-2,y-2), (x+2,y+2), (0,0,0), 1)
cv2.imwrite('result.png', frame)

True

In [77]:
t = time_range[50]
''' READ JSON FILES WITH LABELED BOUNDING BOXES '''
with open(f'cropped_data_labeled/x_vel_cropped_time={np.round(t,1)}0.json', 'r') as f:
    dictData = json.load(f)

In [83]:
frame = cv2.imread(f'cropped_data_labeled/x_vel_cropped_time={np.round(t,1)}0.png')
labeled_mask = np.zeros((frame.shape[:2]), dtype='uint8')
for i, vortex in enumerate(dictData['shapes']):
    a = int(vortex['points'][0][0])
    b = int(vortex['points'][0][1])
    c = int(vortex['points'][1][0])
    d = int(vortex['points'][1][1])
    print(f'vortex {i+1}', a, b, c, d)
    cv2.rectangle(labeled_mask, (a,b), (c,d), (255,255,255), cv2.FILLED)
    cv2.rectangle(frame, (a,b), (c,d), (0,0,0), 1)
cv2.imwrite('result.png', labeled_mask)

vortex 1 235 517 223 505
vortex 2 345 495 358 506
vortex 3 338 333 352 350
vortex 4 695 526 707 536
vortex 5 720 551 709 540
vortex 6 553 749 541 734
vortex 7 511 707 500 694
vortex 8 177 727 188 736
vortex 9 266 700 278 709
vortex 10 132 804 142 812
vortex 11 432 1102 442 1114
vortex 12 512 1058 522 1070
vortex 13 537 968 547 980
vortex 14 732 1010 741 1026
vortex 15 896 898 921 907
vortex 16 741 888 752 902


True

In [90]:
frame.shape[:2]

(1247, 1247)

In [106]:
TP = []
for i, vortex in enumerate(dictData['shapes']):
    labeled_mask = np.zeros((frame.shape[:2]), dtype='uint8')
    a = int(vortex['points'][0][0])
    b = int(vortex['points'][0][1])
    c = int(vortex['points'][1][0])
    d = int(vortex['points'][1][1])
#         print(f'vortex {i+1}', a, b, c, d)
    cv2.rectangle(labeled_mask, (a,b), (c,d), 255, cv2.FILLED)
    for x,y in detected_vortices:
        if (x,y) not in TP:
            detected_vortex = np.zeros((frame.shape[:2]), dtype='uint8')
            cv2.rectangle(detected_vortex, (x-2,y-2), (x+2,y+2), 255, cv2.FILLED)
            result = np.all(cv2.bitwise_and(detected_vortex, labeled_mask) == 0)
            if not result:
                TP.append((x,y))
                break

In [107]:
tp_count = len(TP) #CORRECT
fp_count = len(detected_vortices) - len(TP) #INCORRECT
fn_count = len(dictData['shapes']) - len(TP) #UNDETECTED

In [108]:
tp_count, fp_count, fn_count 

(2, 3, 14)

In [109]:
''' CHECK IF CALCULATION WAS GOOD '''
frame = cv2.imread(f'cropped_data_labeled/x_vel_cropped_time={np.round(t,1)}0.png')
for x,y in TP:
    cv2.rectangle(frame, (x-2,y-2), (x+2,y+2), (0,0,0), 1)
cv2.imwrite('result.png', frame)

True

In [ ]:
detected_vortices[t] = [, , , ]